# SDialog dependencies

In [1]:
%%script false --no-raise-error
# Only run this cell if you are using a non jupyter environment
!conda create --name loco python=3.12 -y
!conda create --name polaco python=3.9 -y
!conda activate loco
!conda activate polaco
!conda install sox
!CFLAGS="-I/Users/yanislabrak/opt/miniconda3/envs/sdialog/include" LDFLAGS="-L/Users/yanislabrak/opt/miniconda3/envs/sdialog/lib" pip install -e dscaper --force-reinstall --no-cache-dir
!pip install -r dscaper/requirements.txt
!pip install -r sdialog/requirements.txt
!pip install -r sdialog/requirements-audio.txt
!pip install numpy==1.26.4

!conda activate jsalt

In [2]:
import os
import json
from tqdm import tqdm

In [3]:
import sdialog
from sdialog import Dialog
from sdialog.generators import PersonaGenerator
from sdialog.personas import Persona, PersonaAgent, Doctor, Patient, Agent

/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Generate Persona

In [4]:
%%script false --no-raise-error
persona_card_folder = "./personas"

sdialog.config.set_llm("aws:anthropic.claude-3-5-sonnet-20240620-v1:0", region_name="us-east-1")

# Generate doctor persona
doctor_persona = Doctor(
    name="Dr. Smith",
    gender="male",
    age=52,
    specialty="Family Medicine"
)
generator_doctor = PersonaGenerator(doctor_persona)
persona_cards = generator_doctor.generate(n=1)
persona_cards.to_file(f"{persona_card_folder}/persona_doctor.json")

# Generate patient persona
patient_persona = Patient(
    name="John Doe",
    gender="male",
    age=62
)
generator_patient = PersonaGenerator(patient_persona)
persona_cards = generator_patient.generate(n=1)
persona_cards.to_file(f"{persona_card_folder}/persona_patient.json")


# Load persona

In [5]:
# Load personas
persona_doctor = Persona.from_file("./personas/persona_doctor.json")
persona_patient = Persona.from_file("./personas/persona_patient.json")

In [6]:
%%script false --no-raise-error
context = "Generate me a 50 turn medical dialogue between patient and doctor, for a primary care visit"

# Create agents
agent1 = PersonaAgent(persona=persona_doctor, name="DOCTOR", dialogue_details=context, response_details="make short turn answers when needed")
agent2 = PersonaAgent(persona=persona_patient, name="PATIENT", dialogue_details=context, response_details="make short turn answers when needed")

In [7]:
save_all = True
GENERATE_PERSONA = True
FORCE_DIALOG_GENERATION = False

os.makedirs("./outputs", exist_ok=True)

In [8]:
if FORCE_DIALOG_GENERATION:

    dialog = agent1.talk_with(agent2, max_turns=3)
    dialog.to_file("dialog_demo.json")

else:
    dialog = Dialog.from_file("dialog_demo.json")

dialog.print()

[dialog_id] 1752861530588
[model] client=<botocore.client.BedrockRuntime object at 0x7fc319ec91c0> model_id='anthropic.claude-3-5-sonnet-20240620-v1:0' region_name='us-east-1' provider='anthropic' supports_tool_choice_values=('auto', 'any', 'tool')
[seed] 226296126
--- Dialogue Begins ---
[DOCTOR] Hello there. I'm Dr. Smith. Welcome to my office. What brings you in to see me today?
[PATIENT] Good morning, Dr. Smith. I'm John Doe. I've been dealing with a persistent cough and feeling pretty tired for about three weeks now. It's starting to wear me down, and I thought I should get it checked out.
[DOCTOR] I'm sorry to hear you've been feeling unwell, Mr. Doe. A persistent cough and fatigue can certainly be troublesome. Let's get some more details. Can you describe your cough? Is it dry or productive?
[PATIENT] Well, Dr. Smith, it's mostly a dry cough. It's particularly bothersome at night when I'm trying to sleep. You know, it reminds me of the time I was teaching about the Civil War and

# Audio dependencies

# Instantiating voices database

In [9]:
from sdialog.audio.voice_database import DummyVoiceDatabase

ModuleNotFoundError: No module named 'sox'

In [ ]:
dummy_voice_database = DummyVoiceDatabase()
print("Instantiating voice database...")

# Instantiating TTS pipeline

In [ ]:
from sdialog.audio.tts_engine import KokoroTTS # ChatterboxTTS, XttsTTS

In [ ]:
tts_pipeline = KokoroTTS()
# tts_pipeline = ChatterboxTTS()
# tts_pipeline = XttsTTS()
print("Instantiating TTS pipeline...")

In [ ]:
from sdialog.audio.audio_dialog import AudioDialog

In [ ]:
# Extend the turns with audio attributes
dialog: AudioDialog = AudioDialog.from_dialog(dialog)

In [ ]:
%%script false --no-raise-error
!git clone https://github.com/cyrta/dscaper.git

In [ ]:
%%script false --no-raise-error
!pip install -e ./dscaper

In [ ]:
import os
import scaper
DATA_PATH = "./dscaper_data"
os.makedirs(DATA_PATH, exist_ok=True)
dsc = scaper.Dscaper(dscaper_base_path=DATA_PATH)

In [ ]:
from sdialog.audio.audio_pipeline import AudioPipeline

In [ ]:
from sdialog.audio.audio_events_enricher import AudioEventsEnricher
enricher = AudioEventsEnricher()

In [ ]:
audio_pipeline = AudioPipeline(
    voice_database=dummy_voice_database,
    tts_pipeline=tts_pipeline,
    dir_audio="./outputs",
    dscaper=dsc,
    enricher = enricher
)

In [ ]:
audio_pipeline.populate_dscaper(["sdialog/BackgroundSound"])

In [ ]:
dialog: AudioDialog = audio_pipeline.inference(dialog, do_word_alignments=False, do_snr=False, do_room_position=False)

In [ ]:
# Enriching the dialog with audio events and generate the timeline of audio events and utterances
dialog = audio_pipeline.enrich(dialog) # TODO: Use the alignment already computed
dialog.timeline.print()
dialog.timeline.draw("./outputs/timeline.png")

## Add SNR and room location

In [ ]:
# TODO: Add SNR and room position to the dialog

# Audio Evaluation

In [ ]:
from sdialog.audio.evaluation import compute_evaluation_utterances, compute_evaluation_audio

In [ ]:
# Utterances level evaluation
metrics_utterances_level = compute_evaluation_utterances(dialog)
for key, value in metrics_utterances_level.items():
    print(f"{key}: {value}")

# Audio level evaluation
metrics_audio_level = compute_evaluation_audio(dialog)
for key, value in metrics_audio_level.items():
    print(f"{key}: {value}")